## Get Setam Auctions

In [ ]:

from bs4 import BeautifulSoup
import pandas as pd
import requests
import time


def get_setam_data(url, vp_num):
    headers = (
        {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36'
         }
    )
    webpage = requests.get(url=url, headers=headers)
    soup = BeautifulSoup(webpage.content, 'html.parser')
    results = soup.find_all(attrs={'class': 'auctions-item'})
    data_results = []

    for result in results:
        title_item = result.find(attrs={'class': 'title-item'})
        item_url = title_item.find('a').attrs['href']
        item_title = title_item.find('h3').text
        annotation_item = result.find(attrs={'class': 'annotation-item'})
        region_item = annotation_item.find(attrs={'class': 'region-item'}).get_text().replace('\n', '').split(': ')
        number_item = annotation_item.find(attrs={'class': 'number-item'}).get_text().replace('\n', '').split(': ')
        start_price_item = annotation_item.find(attrs={'class': 'start-price-item'}).get_text().replace('\n', '').split(': ')
        payment_item = annotation_item.find(attrs={'class': 'payment-item'}).get_text().replace('\n', '').split(': ')
        condition_item = annotation_item.find(attrs={'class': 'condition-item'}).get_text().replace('\n', '').split(': ')
        start_date_item = annotation_item.find(attrs={'class': 'start-date-item'}).get_text().replace('\n', '').split(': ')

        data_result = {
            'АСВП №': vp_num,
            'Заголовок аукціону': item_title,
            'Область': region_item[1],
            'Номер лоту': number_item[1],
            'Початкова ціна': start_price_item[1],
            'Гарантійний внесок': payment_item[1],
            'Стан аукціону': condition_item[1],
            'Дата початку': start_date_item[1],
            'Гіперпосилання': 'https://setam.net.ua{}'.format(item_url),
            }
        data_results.append(data_result)

    return data_results

BASE_URL = "https://setam.net.ua/auctions/filters/provadjenia"
data_results = []

dataset = pd.read_excel("data/vp-test.xlsx",sheet_name="АСВП", engine='openpyxl', na_values='...').fillna('') # engine='openpyxl' ,engine="xlrd"
for row in dataset.itertuples():
    vpOrderNum = row.vpOrderNum
    url = "{}={};".format(BASE_URL, vpOrderNum)
    data = get_setam_data(url=url, vp_num=vpOrderNum)
    data_results.extend(data)
    print("Кількість аукціонів за ВП №{}: {}".format(vpOrderNum, len(data)))
    time.sleep(5)

result_data = pd.DataFrame(data_results)
with pd.ExcelWriter('data/vp-test.xlsx', mode='a', engine='openpyxl', if_sheet_exists='overlay') as writer:
    result_data.to_excel(writer, sheet_name="СЕТАМ", index=False)
# result_data.to_excel('data/setam.xlsx', index=False)
print('Обробку завершено.')


## Save DataFrame to existing Excel File

In [5]:
import pandas as pd


# mydataframe = pd.DataFrame({'Headers': ['Date1', 'Date2'],
#                             'DatesA': ['20.10.2023', '21/10/2023'],
#                             'DatesB': ['20.10.2023', '21/10/2023']
#                             })
# mydataframe["DatesA"] = mydataframe["DatesA"].apply(pd.to_datetime, dayfirst=True)
# print(mydataframe.dtypes)

mydataframe = pd.read_excel("data/vp-test.xlsx",sheet_name="АСВП", engine='openpyxl', na_values='...').fillna('') # engine='openpyxl' ,engine="xlrd"
with pd.ExcelWriter('data/vp-test.xlsx', mode='a', engine='openpyxl', if_sheet_exists='overlay') as writer:
    mydataframe.to_excel(writer, sheet_name="СЕТАМ", index=False)

    # sheet = writer.sheets['СЕТАМ'] # Openpyxl worksheet
    # # sheet["A1"].number_format = 'dd.mm.yyyy'
    # for cell in sheet['B']: # Loop column B 
    #     if cell.row == 1:  # Skip row 1 
    #         continue
    #     ### Apply number format to all cells in Column B to max used row
    #     cell.number_format = 'dd.mm.yyyy' 
